In [1]:
#import necessary packages
import os
import pandas as pd
import re
from collections import Counter
import numpy as np
%pprint

Pretty printing has been turned OFF


In [3]:
# loading in files
directory = "../aps_reviews/"
filenames = os.listdir(directory)
txts = []
for file in filenames:
    with open(directory + file) as f:
        txts.append(f.read())

In [4]:
def title_cleanup(string):
    lowered = string.lower().split('|')[0]
    new = "".join([u for u in lowered if u.isalpha() or u == ' '])
    return " ".join(new.split()[0:6])

In [5]:
def author_cleanup(string):
    lowered = string.lower().split('|')[0]
    no_parens = re.sub("\(.+?\)", "", lowered)
    no_parens = no_parens.rstrip(',')
    no_parens = no_parens.rstrip(' ')
    no_parens = no_parens.replace('.', '')
    no_parens = no_parens.replace('[', '')
    no_parens = no_parens.replace(']', '')
    #split on comma, make first last
    split = no_parens.split(',')
    if type(split) == list:
        head = split[0:1]
        tail = split[1:]

        final = " ".join(tail+head)
    else:
        final = split
    return final 

In [6]:
def author_cleanup_last(string):
    lowered = string.lower().split('|')[0]
    no_parens = re.sub("\(.+?\)", "", lowered)
    no_parens = no_parens.rstrip(',')
    no_parens = no_parens.rstrip(' ')
    no_parens = no_parens.replace('.', '')
    no_parens = no_parens.replace('[', '')
    no_parens = no_parens.replace(']', '')
    #split on comma, make first last
    split = no_parens.split(',')
    if type(split) == list:
        
        return split[0]
    else:
        return split

In [7]:
def author_cleanup_first(string):
    lowered = string.lower().split('|')[0]
    no_parens = re.sub("\(.+?\)", "", lowered)
    no_parens = no_parens.rstrip(',')
    no_parens = no_parens.rstrip(' ')
    no_parens = no_parens.replace('.', '')
    no_parens = no_parens.replace('[', '')
    no_parens = no_parens.replace(']', '')
    #split on comma, make first last
    split = no_parens.split(',')
    if type(split) == list:
        first = split[-1].split()
        try:
            return first[0]
        except:
            return split[-1]
    else:
        return split

In [8]:
#get author and title list
df_pre = pd.read_csv('fiction_metadata.csv', dtype={'oclc':'str','recordid':'str', 'datetype': 'str', 'startdate':'str' }).fillna('')
df_pre = df_pre.loc[df_pre['date'].isin([str(u) for u in range(1878, 1926)])].reset_index(drop=True)

df_post = pd.read_csv('fiction_metadata_post_1923.csv', dtype={'metadatasuspicious':'str'}).fillna('')
df_post = df_post.loc[df_post['inferreddate'].isin([str(u) for u in range(1923, 1928)])].reset_index(drop=True)

df_all = df_pre[['author', 'title', 'date']]
df_post_new = df_post[['author', 'title', 'inferreddate']]
df_post_new.rename(columns={'inferreddate':'date'}, inplace=True)
df_all = df_all.append(df_post_new)

/Users/lavin/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [9]:
#first five/six words, lowercase, remove special characters and punctuation
df_all['title'] = df_all['title'].apply(title_cleanup)
df_all['author_last'] = df_all['author'].apply(author_cleanup_last)
df_all['author_first'] = df_all['author'].apply(author_cleanup_first)
df_all['author_full'] = df_all['author'].apply(author_cleanup)

In [10]:
#dedupe on author, title
df_all = df_all.drop_duplicates(['author_full', 'title']).reset_index(drop=True)

In [11]:
df_all.head()

,author,title,date,author_last,author_first,author_full
0,"McCarthy, Justin,",miss misanthrope,1878,mccarthy,justin,justin mccarthy
1,"Braddon, M. E.",an open verdict,1878,braddon,m,m e braddon
2,"Lee, Holme,",straightforward,1878,lee,holme,holme lee
3,"Cooper, Katherine,",sebastian,1878,cooper,katherine,katherine cooper
4,"Oliphant,",the primrose path,1878,oliphant,oliphant,oliphant


In [12]:
authors_by_last_name = {}

#make a dictionary of lastnames
for row in df_all.iterrows():
    #make the value a list of first_names (ignore any that are one letter)
    if len(row[1]['author_first']) > 2:
        try:
            author = authors_by_last_name[row[1]['author_last']]
            if row[1]['author_first'] not in authors_by_last_name[row[1]['author_last']]:
                if row[1]['author_first'] != row[1]['author_last']:
                    authors_by_last_name[row[1]['author_last']].append(row[1]['author_first'])
        except:
            authors_by_last_name[row[1]['author_last']] = [row[1]['author_first']]

In [13]:
solo_names = {}
for name in authors_by_last_name.keys():
    if len(authors_by_last_name[name]) == 1:
        solo_names[name] = authors_by_last_name[name][0]

In [13]:
#Note: this function needs a lot of work!!!

# 1. titles list is incomplete
# 2. assumes surname will be one word
# 3. returns empty if top last name not in author names dictionary 
# 4. returns empty if the last name is found but the first name isn't
# 5. code to choose the most frequently mentioned last name / first name pair might be coded wrong
# 6. current not check 
# 7. has no fuzziness ... in the example review we have "Lucien Carr", "Mr. Carr", "and "Ldclen Carr"
# So the algorithm could theoretically resolve to Lucien Carr since that's a name and Ldclen isn't
# Lucien Carr isn't in the author names dictionary but we could program this to find top names not in the dict

### eva's edits round 1: trying symspell and names from the us census to correct OCR issues

In [14]:
# first i'm gonna need to combine the documents from every year
name_dir = "../../../Code_stuff/names/"
name_files = os.listdir(name_dir)
len(name_files)

FileNotFoundError: [Errno 2] No such file or directory: '../../../Code_stuff/names/'

In [15]:
name_lists = []
for file in name_files:
    with open(name_dir + file, encoding='latin-1') as f:
        name_lists.extend(f.readlines())

In [16]:
name_lists[:50]

['Emily,F,25956\n', 'Hannah,F,23082\n', 'Madison,F,19968\n', 'Ashley,F,17997\n', 'Sarah,F,17702\n', 'Alexis,F,17629\n', 'Samantha,F,17265\n', 'Jessica,F,15709\n', 'Elizabeth,F,15099\n', 'Taylor,F,15078\n', 'Lauren,F,14175\n', 'Alyssa,F,13553\n', 'Kayla,F,13312\n', 'Abigail,F,13090\n', 'Brianna,F,12879\n', 'Olivia,F,12852\n', 'Emma,F,12552\n', 'Megan,F,11434\n', 'Grace,F,11285\n', 'Victoria,F,10925\n', 'Rachel,F,10675\n', 'Anna,F,10589\n', 'Sydney,F,10242\n', 'Destiny,F,9850\n', 'Morgan,F,9502\n', 'Jennifer,F,9386\n', 'Jasmine,F,9097\n', 'Haley,F,9069\n', 'Julia,F,8766\n', 'Kaitlyn,F,8758\n', 'Nicole,F,8562\n', 'Amanda,F,8554\n', 'Katherine,F,8108\n', 'Natalie,F,8094\n', 'Hailey,F,7830\n', 'Alexandra,F,7547\n', 'Savannah,F,7099\n', 'Chloe,F,7094\n', 'Rebecca,F,7066\n', 'Stephanie,F,7028\n', 'Maria,F,6854\n', 'Sophia,F,6563\n', 'Mackenzie,F,6349\n', 'Allison,F,6314\n', 'Isabella,F,6242\n', 'Mary,F,6198\n', 'Amber,F,6181\n', 'Danielle,F,6088\n', 'Gabrielle,F,5870\n', 'Jordan,F,5807\n']

In [17]:
len(name_lists)

1959378

In [18]:
name_lists[0].split(',')

['Emily', 'F', '25956\n']

In [19]:
name_list = [x[:-1] for x in name_lists if len(x.split(','))==3]

In [20]:
len(name_list)

1957147

In [21]:
name_list[0].split(',')[-1].isnumeric()

True

In [22]:
name_list = [x for x in name_list if x.split(',')[-1].isnumeric()]

In [23]:
len(name_list)

1957047

In [24]:
name_list = [x for x in name_list if x.split(',')[0].isalpha()]

In [25]:
len(name_list)

1957046

In [26]:
name_list = [x for x in name_list if ''.join(x.split(',')).isalnum()]

In [27]:
len(name_list)

1957046

In [28]:
name_df = pd.DataFrame(name_lists, columns = ['name_txt'])

In [29]:
name_df.head()

,name_txt
0,"Emily,F,25956\n"
1,"Hannah,F,23082\n"
2,"Madison,F,19968\n"
3,"Ashley,F,17997\n"
4,"Sarah,F,17702\n"


In [30]:
def justName(name):
    name_parts = name[:-1].split(',')
    return name_parts[0]
def justCount(name):
    name_parts = name[:-1].split(',')
    try:
        return int(name_parts[-1])
    except:
        return 0

In [31]:
name_df['name'] = name_df.name_txt.map(justName)
name_df['freq'] = name_df.name_txt.map(justCount)

In [32]:
name_df.describe()

,freq
count,1.959378e+06
mean,1.801974e+02
std,1.579895e+03
min,0.000000e+00
25%,7.000000e+00
50%,1.200000e+01
75%,3.200000e+01
max,3.160010e+05


In [33]:
name_df = name_df[name_df.freq > 4][['name','freq']]
name_df = name_df[name_df['name'].apply(lambda x: x.isalpha())]

In [34]:
name_df[name_df.name == 'Emily'].aggregate(np.sum)

name    EmilyEmilyEmilyEmilyEmilyEmilyEmilyEmilyEmilyE...
freq                                               852018
dtype: object

In [35]:
len(name_df)

1957046

In [36]:
grouped = name_df.groupby('name').aggregate(np.average)

In [37]:
len(grouped)

98400

In [38]:
grouped['freq'] = grouped.freq.apply(lambda x: round(x))

In [39]:
grouped = grouped.reset_index()

In [40]:
grouped.describe()

,freq
count,98400.000000
mean,28.896951
std,236.364602
min,5.000000
25%,6.000000
50%,7.000000
75%,10.000000
max,18661.000000


In [41]:
grouped.to_csv('name_freq_dict.csv', index=False, header=False)

In [42]:
from symspellpy.symspellpy import SymSpell, Verbosity 

# maximum edit distance per dictionary precalculation
max_edit_distance_dictionary = 2
prefix_length = 7
# create object
sym_spell = SymSpell(max_edit_distance_dictionary, prefix_length)
# load dictionary
dictionary_path = "name_freq_dict.txt"
term_index = 0 
count_index = 1
sym_spell.load_dictionary(dictionary_path, term_index, count_index, separator=",")

True

In [43]:
suggestions = sym_spell.lookup("Ldclen", Verbosity.CLOSEST,
                               max_edit_distance=2, include_unknown=True)

In [44]:
top = suggestions[0]._term

In [45]:
top

'Lucien'

In [46]:
sym_spell._words['Emily']

3981

In [15]:
# test text
mytext = """
    Missouri: A Bone of Contention '                 WE  aware that most of the States of the Union had their nicknames, more or less complimentary, but to name Missouri ' a bone of contention ' is a stroke of wit. It does, however, rightly describe the Missouri of the past, and vividly writes in a phrase her political history. Until the triumph of the Union armies and the close of the Civil War, Missouri was in the jaws of the watch-dogs of slavery and freedom. In war or in peace, the subiect of legislative com-                 promise or of military struggle, Missouri was an uncertain factor. Now, after -five years of national peace, her history may be calmly and impartially written. Indeed, the task has been done, and well done, and the , Lucien Carr of Harvard, may be congratulated upon his work,  is strong, unimpassioned, scholarly, and as impressed with the firm touch which comes of local knowledge as are the imprinted rocks in the cabinets at Cambridge. Long familiarity with the wealth of archaeology in the Peabody Museuml seems to have given him the power of comparison and generalization in the evolution of a commonwealth, while  acquaintance with living men enables him to blend the results of the study and the field in pleasing literary form. Five of his seventeen chapters give a luminous picture of the early French and Spanish discoveries and domination. Then follow three chapters treating of the                 Missouri. By Ldclen Carr. $ti... (American Commonwealths.) Boston:                 Itoughto., Sftfltn & Co.                 l , the compromise, and the  into the U nion of this State named after the great river which flows through it. In his treatment of the period from 1844 to i861, as well as that of war time, some readers may charge Mr. Carr with unduly favoring the Southern and even Confederate view; but to people living this side of the now-vanished Mason and Dixon's line, this is doubtless a benefit; for only when Northern people are able to ' put themselves in the place' of Southerners and see with Southern eyes, can they be sure that they have achieved that impartiality which is essential to the writing of final history. He shows that the Missourians were neither secessionists nor slavery propagandists. He both criticises and justifies the action of the second convention which, in the uncertain hours when other States  seceding and Missouri's Governor had been driven into exile, org  a provisional government, and  saved Missouri Irons ' the pit of political degradation into which the States in rebellion were sunk during the period of reconstruction.' Mr. Carr practically and almost abruptly ends his history at the close of the War, believing that the career of Missouri as a bone of contention ended with the abolition of slavery. The fifty years' struggle was over, the State recovered rapidly froni the wounds of the Civil War, wealth increased wonderfully, and the Negro was liberally dealt with in most if not all points relating to citizenship. Taken as a whole, this book, with its sustained interest, high average literary merit, and thorough treatment of the voluminous facts, fully justifies its place in the series of ' histories of such States as have exercised a positive influence in the shaping of the national Government, or have had a striking political - . . history.' Like the others, it has a good map and index."""  

In [16]:
results = re.findall('dr\. .+? |rev\. .+? |miss .+? |mr\..+? |mrs\..+? ', mytext.lower())

In [17]:
results = [word.replace("'s", "") for word in results]
results

['mr. carr ', 'mr. carr ']

In [18]:
if len(results) > 0:
    top = Counter(results).most_common(1)

In [19]:
top

[('mr. carr ', 2)]

In [52]:
lastname = top[0][0].split('.')[-1].strip()

In [21]:
titles = """Doctor,Dr,Mr,Mrs,Ms,Miss,Msgr,Monsignor,Rev,Reverend,Hon,Honorable,Honourable,Prof,Professor,Madame,Madam,Lady,Lord,Sir,Dame,Master,Mistress,Chancellor,Principal,President,Pres,Warden,Dean,Regent,Rector,Provost,Director
"""

In [22]:
titles = titles.lower().rstrip().split(',')

In [23]:
titles

['doctor', 'dr', 'mr', 'mrs', 'ms', 'miss', 'msgr', 'monsignor', 'rev', 'reverend', 'hon', 'honorable', 'honourable', 'prof', 'professor', 'madame', 'madam', 'lady', 'lord', 'sir', 'dame', 'master', 'mistress', 'chancellor', 'principal', 'president', 'pres', 'warden', 'dean', 'regent', 'rector', 'provost', 'director']

In [56]:
title_list = r"\.? .+? |".join(titles)

In [57]:
title_list

'doctor\\.? .+? |dr\\.? .+? |mr\\.? .+? |mrs\\.? .+? |ms\\.? .+? |miss\\.? .+? |msgr\\.? .+? |monsignor\\.? .+? |rev\\.? .+? |reverend\\.? .+? |hon\\.? .+? |honorable\\.? .+? |honourable\\.? .+? |prof\\.? .+? |professor\\.? .+? |madame\\.? .+? |madam\\.? .+? |lady\\.? .+? |lord\\.? .+? |sir\\.? .+? |dame\\.? .+? |master\\.? .+? |mistress\\.? .+? |chancellor\\.? .+? |principal\\.? .+? |president\\.? .+? |pres\\.? .+? |warden\\.? .+? |dean\\.? .+? |regent\\.? .+? |rector\\.? .+? |provost\\.? .+? |director'

In [59]:
results=[]
for x in titles:
    query = r"\.? .+? "
    print(re.findall(x+query, mytext.lower()))


[]
[]
['mr. carr ', 'mr. carr ']
[]
['ms to ']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [60]:
re.findall(title_list, mytext.lower())






['ms to ', 'mr. carr ', 'mr. carr ']

In [68]:
def findLastName(text):
    """
    Returns words following common 
    """
    results = re.findall('dr\. .+? |rev\. .+? |miss .+? |mr\..+? |mrs\..+? ', text.lower())
    results = [word.replace("'s", "") for word in results]
    if len(results) > 0:
        top = Counter(results).most_common(1)
        return top[0][0].split('.')[-1].strip()
    else:
        raise Exception('no last names found')

In [69]:
findLastName(mytext)

'carr'

In [ ]:
def findFirstName(text, lastname):
    results = re.findall('dr\. .+? |rev\. .+? |miss .+? |mr\..+? |mrs\..+? ', text.lower())

In [139]:
first_name_results = []

au_first_names = authors_by_last_name[lastname]
            
for au_name in au_first_names:
    first_name_matches = re.search(au_name, mytext.lower())
    if first_name_matches:
        first_name_results.append(au_name)

In [140]:
au_first_names

['alice', 'clark', 'sarah', 'john', 'annie']

In [137]:
def find_top_author(text, name_dict):
    """ This function looks at a block of text, possibly a book review, and 
    uses a dictionary of names to find the top first name and last name pair 
    mentioned in the text. A statistical method for name inference would probably 
    be preferable to this method"""

    #get the most frequent mr or mrs., miss, dr in the review 
    #and all the characters after the honorific until the next space (non-greedy)    
    results = re.findall('dr\. .+? |rev\. .+? |miss .+? |mr\..+? |mrs\..+? ', text.lower())
    
    #remove possessives
    results = [word.replace("'s", "") for word in results]
    print(results)

    if len(results) > 0:
        top = Counter(results).most_common(1)

        lastname = top[0][0].split('.')[-1].strip()
            
        #get authors with same last name
        first_name_results = []
        try:
            au_first_names = authors_by_last_name[lastname]
            
            for au_name in au_first_names:
                first_name_matches = re.search(au_name, text.lower())
                if first_name_matches:
                    first_name_results.append(au_name)
            if len(first_name_results) > 0:
                topfirst = Counter(first_name_results).most_common(1)
                
                firstname = topfirst[0][0]
                
            else:
                firstname = ''
        except:
            firstname = ''
        if firstname == '':
            for i in both:
                first_name_matches = re.search(i, text.lower())
                if first_name_matches:
                    first_name_results.append(i)
            
        #match a first name or pass
        if len(lastname) > 4:
            return [lastname, firstname]
        else:
            return ["no match", "no match"]

In [138]:
print(find_top_author(mytext, authors_by_last_name))

['mr. carr ', 'mr. carr ']


NameError: name 'both' is not defined

### eva's edits round 2: trying to find words commonly preceding authors names

In [168]:
df_all.head()

,author,title,date,author_last,author_first,author_full
0,"McCarthy, Justin,",miss misanthrope,1878,mccarthy,justin,justin mccarthy
1,"Braddon, M. E.",an open verdict,1878,braddon,m,m e braddon
2,"Lee, Holme,",straightforward,1878,lee,holme,holme lee
3,"Cooper, Katherine,",sebastian,1878,cooper,katherine,katherine cooper
4,"Oliphant,",the primrose path,1878,oliphant,oliphant,oliphant


In [174]:
txts[0][:300].lower().strip()

'literary notes.                 -the chautauqua department of " wide awake" is now published separately.                 -mr. charles dudley warner has written for"the north american review " a paper on p'

In [197]:
last_names = list(set(authors_by_last_name.keys()))[1:]

In [198]:
last_names[0]

"o'hanrahan"

In [180]:
'|'.join(titles)

' doctor|dr|mr|mrs|ms|miss|msgr|monsignor|m|rev|reverend|hon|honorable|honourable|prof|professor|madame|madam|lady|lord|sir|dame|master|mistress|chancellor|principal|president|pres|warden|dean|regent|rector|provost|director'

In [238]:
title_poss = title_list + r". (.+)*?"

In [239]:
title_poss

'(doctor|dr|mr|mrs|ms|miss|msgr|monsignor|m|rev|reverend|hon|honorable|honourable|prof|professor|madame|madam|lady|lord|sir|dame|master|mistress|chancellor|principal|president|pres|warden|dean|regent|rector|provost|director). (.+)*?'

In [240]:
title_last_names = [title_poss+name for name in last_names]

In [241]:
title_last_names[-1]

'(doctor|dr|mr|mrs|ms|miss|msgr|monsignor|m|rev|reverend|hon|honorable|honourable|prof|professor|madame|madam|lady|lord|sir|dame|master|mistress|chancellor|principal|president|pres|warden|dean|regent|rector|provost|director). (.+)*?funck-brentano'

In [242]:
'|'.join(title_last_names)[:500]

"(doctor|dr|mr|mrs|ms|miss|msgr|monsignor|m|rev|reverend|hon|honorable|honourable|prof|professor|madame|madam|lady|lord|sir|dame|master|mistress|chancellor|principal|president|pres|warden|dean|regent|rector|provost|director). (.+)*?o'hanrahan|(doctor|dr|mr|mrs|ms|miss|msgr|monsignor|m|rev|reverend|hon|honorable|honourable|prof|professor|madame|madam|lady|lord|sir|dame|master|mistress|chancellor|principal|president|pres|warden|dean|regent|rector|provost|director). (.+)*?fenollosa|(doctor|dr|mr|mrs"

In [243]:
pattern = '|'.join(title_last_names)

In [245]:
re.search(pattern,mytext)

KeyboardInterrupt: 

In [207]:
for doc in [txt.lower().strip() for txt in txts]:
    results = re.findall(pattern,doc)
    print(results[0])

('', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'm', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''

('', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'miss', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',

KeyboardInterrupt: 

In [ ]:
authors_by_last_name[lastname]